In [1]:
#!pip install -qq transformers genomic-benchmarks datasets

In [2]:
### Parameters
MODEL_NAME = "simecek/DNADeberta2"
TOKENIZER_NAME = "armheb/DNA_bert_6"
K = 6
STRIDE = K

# if less than 1, only this fraction of each dataset is used
DATASET_THINING = 0.01

BENCHMARKS_FOLDER = '/root/.genomic_benchmarks'
# BENCHMARKS_FOLDER = '/home/jovyan/.genomic_benchmarks/' (for INFRA HUB)

DATASETS = [('demo_coding_vs_intergenomic_seqs', 0),
 ('demo_human_or_worm', 0), ('human_enhancers_cohn', 0), ('human_enhancers_ensembl', 0),
 ('human_ensembl_regulatory', 0), ('human_nontata_promoters', 0), ('human_ocr_ensembl', 0)]

BATCH_SIZE = 32
LEARNING_RATE = 8e-5
EPOCHS = 3
RUNS = 3

# do not forget to attach drive
OUTPUT_PATH = 'drive/MyDrive/genomic_benchmarks/experiment1.csv'

## Download benchmark datasets and tokenizer

In [3]:
from genomic_benchmarks.loc2seq import download_dataset
from genomic_benchmarks.data_check.info import is_downloaded
from pathlib import Path
from tqdm.autonotebook import tqdm

for dataset_name, dataset_version in tqdm(DATASETS):
    if not is_downloaded(dataset_name):
        download_dataset(dataset_name, version=dataset_version)

benchmark_root = Path(BENCHMARKS_FOLDER)

/usr/local/lib/python3.7/dist-packages/genomic_benchmarks/utils/datasets.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


  0%|          | 0/7 [00:00<?, ?it/s]

Downloading...
From: https://drive.google.com/uc?id=1cpXg0ULuTGF7h1_HTYvc6p8M-ee43t-v
To: /root/.genomic_benchmarks/demo_coding_vs_intergenomic_seqs.zip

  0%|          | 0.00/33.9M [00:00<?, ?B/s]
100%|██████████| 33.9M/33.9M [00:00<00:00, 197MB/s]
Downloading...
From: https://drive.google.com/uc?id=1JW0-eTB-rJXvFcglqBo3pFZi1kyIWC3X
To: /root/.genomic_benchmarks/demo_human_or_worm.zip

  0%|          | 0.00/28.9M [00:00<?, ?B/s]
100%|██████████| 28.9M/28.9M [00:00<00:00, 145MB/s]
Downloading...
From: https://drive.google.com/uc?id=176563cDPQ5Y094WyoSBF02QjoVQhWuCh
To: /root/.genomic_benchmarks/human_enhancers_cohn.zip

  0%|          | 0.00/11.9M [00:00<?, ?B/s]
100%|██████████| 11.9M/11.9M [00:00<00:00, 52.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1gZBEV_RGxJE8EON5OObdrp5Tp8JL0Fxb
To: /root/.genomic_benchmarks/human_enhancers_ensembl.zip

  0%|          | 0.00/51.1M [00:00<?, ?B/s]
  9%|▉         | 4.72M/51.1M [00:00<00:02, 15.7MB/s]
100%|██████████| 51.1M/51.1M [00:

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_NAME)

Downloading:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [5]:
def kmers_strideK(s, k=K):
    return [s[i:i + k] for i in range(0, len(s), k) if i + k <= len(s)]

def kmers_stride1(s, k=K):
    return [s[i:i + k] for i in range(0, len(s)-k+1)]

if (STRIDE == 1):
  kmers = kmers_stride1
else:
  kmers = kmers_strideK

# function used for the actual tokenization
def tok_func(x): return tokenizer(" ".join(kmers(x["seq"])))

# example
tok_func({'seq': 'ATGGAAAGAGGCACCATTCT'})    

{'input_ids': [2, 501, 835, 650, 3], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}

## Looping through datasets, fine-tuning the model for each of them, logging metrics

In [6]:
import pandas as pd
import numpy as np
from random import random, randrange
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import Dataset, DatasetDict, load_metric

def compute_metrics_binary(eval_preds):
    metric = load_metric("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

def compute_metrics_multi(eval_preds):
    metric = load_metric("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

outputs = []

for dataset_name, dataset_version in tqdm(DATASETS):
    

    labels = sorted([x.stem for x in (benchmark_root / dataset_name / 'train').iterdir()])

    tmp_dict = {}

    for split in ['train', 'test']:
        for nlabel, label in enumerate(labels):
            for f in (benchmark_root / dataset_name / split / label).glob('*.txt'):
                txt = f.read_text()
                if not DATASET_THINING or DATASET_THINING==1:
                    tmp_dict[f"{label} {f.stem}"] = (split, nlabel, txt)
                elif random() < DATASET_THINING:
                    tmp_dict[f"{label} {f.stem}"] = (split, nlabel, txt)

    df = pd.DataFrame.from_dict(tmp_dict).T.rename(columns = {0: "dset", 1: "cat", 2: "seq"})

    ds = Dataset.from_pandas(df)

    tok_ds = ds.map(tok_func, batched=False, remove_columns=['__index_level_0__', 'seq'])
    tok_ds = tok_ds.rename_columns({'cat':'labels'})

    dds = DatasetDict({
        'train': tok_ds.filter(lambda x: x["dset"] == "train").remove_columns('dset'),
        'test':  tok_ds.filter(lambda x: x["dset"] == "test").remove_columns('dset')
    })

    compute_metrics = compute_metrics_binary if len(labels) == 2 else compute_metrics_multi

    for _ in range(RUNS):

        model_cls = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(labels))

        args = TrainingArguments('outputs', learning_rate=LEARNING_RATE, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
            evaluation_strategy="epoch", per_device_train_batch_size=BATCH_SIZE, per_device_eval_batch_size=BATCH_SIZE*2,
            num_train_epochs=EPOCHS, weight_decay=0.01, save_steps=100000, seed=randrange(1,10001), report_to='none')
        
        trainer = Trainer(model_cls, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                          tokenizer=tokenizer, compute_metrics=compute_metrics)
        trainer.train()
        
        max_accuracy = max([x['eval_accuracy'] for x in trainer.state.log_history if 'eval_accuracy' in x])
        max_f1 = max([x['eval_f1'] for x in trainer.state.log_history if 'eval_f1' in x]) if len(labels) == 2 else np.nan
        train_runtime = max([x['train_runtime'] for x in trainer.state.log_history if 'train_runtime' in x])
        
        outputs.append((dataset_name, max_accuracy, max_f1, train_runtime))




  0%|          | 0/7 [00:00<?, ?it/s]

Parameter 'function'=<function tok_func at 0x7f10b298df80> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/1037 [00:00<?, ?ex/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

...

***** Running Evaluation *****
  Num examples = 369
  Batch size = 64
***** Running Evaluation *****
  Num examples = 369
  Batch size = 64
***** Running Evaluation *****
  Num examples = 369
  Batch size = 64


Training completed. Do not forget to share your model on huggingface.co/models =)


loading configuration file https://huggingface.co/simecek/DNADeberta2/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/eaa84554db67bd0bd1237534157006a6e5a26745104c342ca1a8117c146aa517.c7f87ccb839059561ef6996c9f8559644ce254107c992b74bd0b46fbe8c80d51
Model config DebertaConfig {
  "_name_or_path": "simecek/DNADeberta2",
  "architectures": [
    "DebertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta",
  "num_atten

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.686425,0.560976,0.463576
2,No log,0.678976,0.609756,0.610811
3,No log,0.740355,0.623306,0.601719


***** Running Evaluation *****
  Num examples = 369
  Batch size = 64
***** Running Evaluation *****
  Num examples = 369
  Batch size = 64
***** Running Evaluation *****
  Num examples = 369
  Batch size = 64


Training completed. Do not forget to share your model on huggingface.co/models =)


loading configuration file https://huggingface.co/simecek/DNADeberta2/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/eaa84554db67bd0bd1237534157006a6e5a26745104c342ca1a8117c146aa517.c7f87ccb839059561ef6996c9f8559644ce254107c992b74bd0b46fbe8c80d51
Model config DebertaConfig {
  "_name_or_path": "simecek/DNADeberta2",
  "architectures": [
    "DebertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta",
  "num_atten

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.691714,0.574526,0.643991
2,No log,0.753040,0.615176,0.630208
3,No log,0.877173,0.593496,0.531250


***** Running Evaluation *****
  Num examples = 369
  Batch size = 64
***** Running Evaluation *****
  Num examples = 369
  Batch size = 64
***** Running Evaluation *****
  Num examples = 369
  Batch size = 64


Training completed. Do not forget to share your model on huggingface.co/models =)




## Outputs

In [7]:
outputs_df = pd.DataFrame(outputs, columns = ['dataset', 'accuracy', 'f1', 'train_runtime'])
outputs_df

,dataset,accuracy,f1,train_runtime
0,demo_coding_vs_intergenomic_seqs,0.887967,0.888889,6.2117
1,demo_coding_vs_intergenomic_seqs,0.896266,0.899598,5.6200
2,demo_coding_vs_intergenomic_seqs,0.879668,0.879032,5.7523
3,demo_human_or_worm,0.931452,0.931727,5.5474
4,demo_human_or_worm,0.947581,0.947791,5.6128
5,demo_human_or_worm,0.927419,0.928571,5.5271
6,human_enhancers_cohn,0.645570,0.562500,2.7860
7,human_enhancers_cohn,0.683544,0.615385,2.8319
8,human_enhancers_cohn,0.696203,0.666667,2.7412
9,human_enhancers_ensembl,0.705502,0.740157,8.9090


In [8]:
outputs_df.groupby('dataset').agg({'accuracy' : ['mean', 'sem'], 'f1' : ['mean','sem'], 'train_runtime': ['mean', 'sem']})

accuracy                  f1            \
                                      mean       sem      mean       sem   
dataset                                                                    
demo_coding_vs_intergenomic_seqs  0.887967  0.004791  0.889173  0.005939   
demo_human_or_worm                0.935484  0.006159  0.936030  0.005951   
human_enhancers_cohn              0.675105  0.015213  0.614850  0.030072   
human_enhancers_ensembl           0.707659  0.001079  0.742263  0.009322   
human_ensembl_regulatory          0.872138  0.005909       NaN       NaN   
human_nontata_promoters           0.813084  0.005396  0.808629  0.007623   
human_ocr_ensembl                 0.634146  0.015089  0.643148  0.018431   

                                 train_runtime            
                                          mean       sem  
dataset                                                   
demo_coding_vs_intergenomic_seqs      5.861333  0.179298  
demo_human_or_worm                    5.562433  0.025856  
human_enhancers_cohn                  2.786367  0.026183  
human_enhancers_ensembl               8.909233  0.000561  
human_ensembl_regulatory             19.426133  0.083330  
human_nontata_promoters               2.910867  0.068478  
human_ocr_ensembl                     9.966333  0.029870

In [11]:
# saving outputs to csv file
outputs_df.to_csv(OUTPUT_PATH, index=False)